In [1]:
import polars as pl
import polars_ds as pds
# Requires version >= v0.5.1
print(pds.__version__)

0.5.1


In [2]:
size = 50_000
df = pds.random_data(size=size, n_cols=0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
    pds.random_int(0, 3).alias("categories"),
    id = pl.Series(values=list(range(size)))
).with_columns(
    pl.col("id").cast(pl.UInt32),
    y = pl.col("x1") * 0.5 + pl.col("x2") * 0.25 - pl.col("x3") * 0.15 + pds.random() * 0.0001,
)

In [3]:
%%timeit
df.select(
    pds.query_lstsq(
        "x1", "x2", "x3",
        target = "y",
        method = "l2",
        l2_reg = 0.1,
    )
)

347 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
df_x = df.select("x1", "x2", "x3").to_pandas()
df_y = df.select("y").to_pandas()

In [5]:
%%timeit
reg = Ridge(alpha = 0.1, fit_intercept=False)
reg.fit(df_x, df_y)

1.45 ms ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
df.select(
    pds.query_knn_ptwise(
        "x1", "x2", "x3",
        index = "id",
        dist = "l1",
        k = 2,
        epsilon = 0.1
    )
)